In [ ]:
pwd

In [ ]:
cd Rar

In [ ]:
cd object-detection-nn

In [ ]:
# cd ../../..

In [ ]:
remote_mode = True

import os
import numpy as np
from nn.YOLO_VGG16_OBB.utils.constants import ANCHORS
from nn.YOLO_VGG16_OBB.prepare_data.dota_dataset_memory import DotaDataset
from nn.YOLO_VGG16_OBB.prepare_data.transforms import train_transform, test_transform
from nn.YOLO_VGG16_OBB.utils.helpers import convert_cells_to_bboxes, load_checkpoint, nms, plot_image, save_checkpoint
from nn.YOLO_VGG16_OBB.utils.constants import device, s, leanring_rate, save_model, checkpoint_file
from nn.YOLO_VGG16_OBB.model.YOLO_VGG16_OBB import YOLO_VGG16_OBB
import torch
import torch.optim as optim
from nn.YOLO_VGG16_OBB.model.loss import YOLOLoss
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as T

if remote_mode:
    model_path_base = f"/home/dcor/niskhizov/Rar/object-detection-nn/nn/YOLO_VGG16_OBB/notebooks/vgg_f_obb_model"
else:
    model_path_base = f"nn/YOLO_VGG16_OBB/notebooks/"


In [ ]:
categories = ['plane','ship', 'storage-tank', 'baseball-diamond', 'tennis-court', 'basketball-court', 'ground-track-field', 'harbor', 'bridge', 'large-vehicle', 'small-vehicle', 'helicopter', 'roundabout', 'soccer-ball-field', 'swimming-pool']


In [ ]:
# Creating the model from YOLOv3 class 
load_model = True
save_model = True
model = YOLO_VGG16_OBB(num_classes=len(categories)).to(device) 

# Defining the optimizer 
optimizer = optim.Adam(model.parameters(), lr = leanring_rate) 

# Defining the loss function 
loss_fn = YOLOLoss() 

# Defining the scaler for mixed precision training 
scaler = torch.amp.GradScaler(device=device) 
# Loading the checkpoint 
if load_model: 
    load_checkpoint(model_path_base + f"e2700_vgg16_{checkpoint_file}", model, optimizer, leanring_rate, device) 

# Initialize TensorBoard writer
writer = SummaryWriter(log_dir='runs/YOLO_VGG16_OBB_v2')



In [ ]:
dataset = DotaDataset( 
	categories=categories,
	grid_sizes=[13, 26, 52], 
	anchors=ANCHORS, 
	transform=train_transform 
) 

# Defining the train data loader 
train_loader = torch.utils.data.DataLoader( 
	dataset=dataset, 
	batch_size=2, 
	shuffle=True, 
	num_workers=2,
 	prefetch_factor=10,
) 


In [ ]:
val_dataset = DotaDataset(
    categories=categories,
    grid_sizes=[13, 26, 52],
    anchors=ANCHORS,
    transform=test_transform,  # Use the same transform for validation
    data_base_path = f"nn/dotadataset/train"
)

# Create the validation data loader
val_loader = torch.utils.data.DataLoader(
    dataset=val_dataset,
    batch_size=2,
    shuffle=True,
)

val_loader_iter = iter(val_loader)

In [ ]:
# Scaling the anchors 
scaled_anchors = ( 
	torch.tensor(ANCHORS) *
	torch.tensor(s).unsqueeze(1).unsqueeze(1).repeat(1,3,2) 
).to(device) 

In [ ]:
val_loader_iter = iter(val_loader)
x, y = next(val_loader_iter)


In [ ]:
loss_fn = YOLOLoss()

In [ ]:
e

In [ ]:
epochs = 100000000
error_counter = 0
save_model = True
# Training the model 
for e in range(3142, epochs+1): 
	print("Epoch:", e) 
    ################# dos
    # Creating a progress bar 
	progress_bar = tqdm(train_loader, leave=True) 

	# Initializing a list to store the losses 
	losses = [] 
	try:
		# Iterating over the training data 
		for batch_idx, (x, y) in enumerate(progress_bar):
			# try:
				print("batch_idx:", batch_idx)
				x = x.to(device) 
				y0, y1, y2 = ( 
					y[0].to(device), 
					y[1].to(device), 
					y[2].to(device), 
				) 

				with torch.amp.autocast(device_type=device): 
					# Getting the model predictions 
					outputs = model(x) 
					# Calculating the loss at each scale 
					loss = ( 
						loss_fn(outputs[0], y0, scaled_anchors[0]) 
						+ loss_fn(outputs[1], y1, scaled_anchors[1]) 
						+ loss_fn(outputs[2], y2, scaled_anchors[2]) 
					) 

				# Add the loss to the list 
				losses.append(loss.item()) 

				# Reset gradients 
				optimizer.zero_grad() 

				# Backpropagate the loss 
				scaler.scale(loss).backward() 

				# Optimization step 
				scaler.step(optimizer) 

				# Update the scaler for next iteration 
				scaler.update() 

				# update progress bar with loss 
				mean_loss = sum(losses) / len(losses) 
				progress_bar.set_postfix(loss=mean_loss)
		
				# Log the loss to TensorBoard
				writer.add_scalar('Loss/train', mean_loss, e * len(train_loader) + batch_idx)

				# # Log images to TensorBoard every 100 batches
				# if batch_idx % 200 == 0:
				# 	# Saving the model 
				# 	if save_model: 
				# 		save_checkpoint(model, optimizer, filename=model_path_base +f"b{batch_idx}_vgg16_checkpoint.pth.tar")
		
				# 	for i in range(0, batch_idx-3, 200):
				# 			if os.path.exists(model_path_base + f"b{i}_vgg16_checkpoint.pth.tar"):
				# 				os.remove(model_path_base + f"b{i}_vgg16_checkpoint.pth.tar")

		if e % 100 == 0:
			# Saving the model 
			if save_model: 
				save_checkpoint(model, optimizer, filename=model_path_base +f"e{e}_vgg16_checkpoint.pth.tar")
				# delete batch checkpoints
				for i in range(0, batch_idx+1, 200):
					if os.path.exists(model_path_base + f"b{i}_vgg16_checkpoint.pth.tar"):
						os.remove(model_path_base + f"b{i}_vgg16_checkpoint.pth.tar")
				# for i in range(0,e-3):
				# 	if os.path.exists(model_path_base + f"e{i}_vgg16_checkpoint.pth.tar"):
				# 		os.remove(model_path_base + f"e{i}_vgg16_checkpoint.pth.tar")
		
			model.eval()
		
			# Getting a sample image from the test data loader 
			try:
				x, y = next(val_loader_iter)
			except StopIteration:
				val_loader_iter = iter(val_loader)
				x, y = next(val_loader_iter)
			x = x.to(device) 
			
			print("###################################### display and report image ######################################")
			with torch.no_grad():
				scaled_anchors = ( 
				torch.tensor(ANCHORS) *
				torch.tensor(s).unsqueeze(1).unsqueeze(1).repeat(1,3,2) 
				).to(device) 
				output = model(x)
				y0, y1, y2 = ( 
					y[0].to(device), 
					y[1].to(device), 
					y[2].to(device), 
				) 

				with torch.amp.autocast(device_type=device): 
					# Getting the model predictions 
					outputs = model(x) 
					# Calculating the loss at each scale 
					loss = ( 
						loss_fn(outputs[0], y0, scaled_anchors[0]) 
						+ loss_fn(outputs[1], y1, scaled_anchors[1]) 
						+ loss_fn(outputs[2], y2, scaled_anchors[2]) 
					) 
		
				# Print predictions
				writer.add_scalar('Loss/val', loss.item(), e * len(train_loader) + batch_idx)

				# bboxes = [[] for _ in range(x.shape[0])]
				# for i in range(3):
				# 	batch_size, A, S, _, _ = output[i].shape
				# 	anchor = scaled_anchors[i]
				# 	boxes_scale_i = convert_cells_to_bboxes(output[i], anchor, s=S, is_predictions=True)
				# 	for idx, box in enumerate(boxes_scale_i):
				# 		bboxes[idx] += box
		
				# i = 0
				# print('bboxes[i] shape:', np.array(bboxes[i]).shape)
				# nms_boxes = nms(bboxes[i], iou_threshold=0.5, threshold=0.6)
				# img_with_boxes = plot_image(x[i].permute(1, 2, 0).detach().cpu(), nms_boxes, categories)
				# img_with_boxes = T.ToTensor()(img_with_boxes)
				# writer.add_image(f'Val/Image_{e}_{i}_{batch_idx}_before', img_with_boxes, e * len(train_loader) + batch_idx)

			model.train()
	except Exception as error:
		print(error)
		error_counter += 1
		if error_counter > 10:
			raise error


    #################
	# training_loop(train_loader, model, optimizer, loss_fn, scaler, scaled_anchors) 


In [ ]:
e

In [ ]:
nms_boxes = nms(bboxes[i], iou_threshold=0.5, threshold=0.6)

In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
import numpy as np
from nn.YOLO_VGG16_OBB.utils.constants import device
# Defining a function to calculate Intersection over Union (IoU)


def iou(box1, box2, is_pred=True):
    if is_pred:
        # IoU score for prediction and label
        # box1 (prediction) and box2 (label) are both in [x, y, width, height, angle] format
        # Convert boxes to polygons
        polys1 = []
        polys2 = []

        # angle (box1[..., 4]) is in radian than- convert to degree
        angle1 = box1[..., 4] * (torch.pi / 2)
        angle2 = box2[..., 4] * (torch.pi / 2)
        angle1_degree = torch.rad2deg(angle1)
        angle2_degree = torch.rad2deg(angle2)

        for i in range(box1.shape[0]):
            poly1 = cv2.boxPoints(((box1[i, 0].item(), box1[i, 1].item(
            )), (box1[i, 2].item(), box1[i, 3].item()), angle1_degree[i].item()))
            poly2 = cv2.boxPoints(((box2[i, 0].item(), box2[i, 1].item(
            )), (box2[i, 2].item(), box2[i, 3].item()), angle2_degree[i].item()))
            polys1.append(poly1)
            polys2.append(poly2)

        # Convert polygons to torch tensors
        poly1 = np.array(polys1, dtype=np.float32)
        poly2 = np.array(polys2, dtype=np.float32)
        # Calculate intersection area
        inter_area = polygon_intersection_area(poly1, poly2)
        # Calculate union area
        box1_area = box1[..., 2] * box1[..., 3]
        box2_area = box2[..., 2] * box2[..., 3]
        union_area = box1_area + box2_area - inter_area

        # Calculate IoU score
        epsilon = 1e-6
        iou_score = inter_area / (union_area + epsilon)

        # Return IoU score
        return iou_score.unsqueeze(1)

    else:
        # IoU score based on width and height of bounding boxes

        # Calculate intersection area
        intersection_area = torch.min(
            box1[..., 0], box2[..., 0]) * torch.min(box1[..., 1], box2[..., 1])

        # Calculate union area
        box1_area = box1[..., 0] * box1[..., 1]
        box2_area = box2[..., 0] * box2[..., 1]        
        union_area = box1_area + box2_area - intersection_area

        # Calculate IoU score
        iou_score = intersection_area / union_area

        # Return IoU score
        return iou_score.unsqueeze(1)



def polygon_intersection_area(poly1_np, poly2_np):
    # Ensure tensors are on CPU and convert to NumPy
    # poly1_np = poly1.detach().cpu().numpy().astype(np.float32)
    # poly2_np = poly2.detach().cpu().numpy().astype(np.float32)

    inter_areas = []
    for p1, p2 in zip(poly1_np, poly2_np):
        inter_poly = cv2.intersectConvexConvex(p1, p2)
        if inter_poly[0] > 0 and inter_poly[1] is not None:
            inter_areas.append(cv2.contourArea(inter_poly[1]))
        else:
            inter_areas.append(0.0)
    inter_areas = np.array(inter_areas, dtype=np.float32)
    # Convert result back to a tensor
    return torch.tensor(inter_areas, dtype=torch.float32, device=device)


def nms(bboxes_orig, iou_threshold, threshold):
    # Filter out bounding boxes with confidence below the threshold.
    bboxes = [box for box in bboxes_orig if box[1] > threshold]

    # Sort the bounding boxes by confidence in descending order.
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)

    # Initialize the list of bounding boxes after non-maximum suppression.
    if (len(bboxes) > 0):
        first_box = bboxes.pop(0)
        bboxes_nms = [first_box]
    else:
        bboxes_nms = [max(bboxes_orig, key=lambda x: x[1])]

    while len(bboxes) >= 0:
        # Iterate over the remaining bounding boxes.
        for box in bboxes:
            # If the bounding boxes do not overlap or if the first bounding box has
            # a higher confidence, then add the second bounding box to the list of
            # bounding boxes after non-maximum suppression.
            if box[0] != first_box[0] or iou(
                    torch.tensor([first_box[2:]], device=device),
                    torch.tensor([box[2:]], device=device),
            ) < iou_threshold:
                # Check if box is not in bboxes_nms
                if box not in bboxes_nms:
                    # Add box to bboxes_nms
                    bboxes_nms.append(box)

        # Get the first bounding box.
        if len(bboxes) > 0:
            first_box = bboxes.pop(0)
        else:
            break

    # Return bounding boxes after non-maximum suppression.
    # box concist: [class_pred, score, x, y, width, height, angle]
    return bboxes_nms

In [ ]:
import torch.nn as nn

sigmoid = nn.Sigmoid()
sigmoid(outputs[0][0][..., 0:1])

In [ ]:
import torch
import torch.nn as nn

from nn.YOLO_VGG16_OBB.utils.helpers import iou

# Defining YOLO loss class


class YOLOLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()
        self.cross_entropy = nn.CrossEntropyLoss()
        self.sigmoid = nn.Sigmoid()
        self.than = nn.Tanh()

    def forward(self, pred, target, anchors):
        # Identifying which cells in target have objects
        # and which have no objects
        obj = target[..., 0] == 1
        no_obj = target[..., 0] == 0

        # Calculating No object loss
        no_object_loss = 10 * self.bce(
            self.sigmoid(pred[..., 0:1][no_obj]), (target[..., 0:1][no_obj]),
        )
        
        # reg_object_loss = 10 * self.bce(
        #     self.sigmoid(pred[..., 0:1][obj]), (target[..., 0:1][obj]),
        # )

        # Reshaping anchors to match predictions
        anchors = anchors.reshape(1, 3, 1, 1, 2)
        pred_angle = torch.tanh(pred[..., 5])
        # Box prediction confidence
        box_preds = torch.cat([self.sigmoid(pred[..., 1:3]),
                               torch.exp(pred[..., 3:5]) *
                               anchors, pred_angle.unsqueeze(-1)
                               ], dim=-1)
        # Calculating intersection over union for prediction and target
        ious = iou(box_preds[obj], target[..., 1:6][obj]).detach()
        # Calculating Object loss
        object_loss = 10 * self.mse(self.sigmoid(pred[..., 0:1][obj]),
                               ious * target[..., 0:1][obj]).clamp(0, 10)

        # Predicted box coordinates
        pred[..., 1:3] = self.sigmoid(pred[..., 1:3])
        # Target box coordinates
        target[..., 3:5] = torch.log(1e-6 + target[..., 3:5] / anchors)
        # Calculating box coordinate loss

        box_loss = self.mse(pred[..., 1:5][obj],
                            target[..., 1:5][obj])

        # Calculate angle loss
        angle_loss = 10 * self.mse(pred_angle[obj],
                              target[..., 5][obj])
        # Claculating class loss
        class_loss = self.cross_entropy((pred[..., 6:][obj]),
                                        target[..., 6][obj].long())

        # print('~~~~~~~~~~~~~ FIND UNCAPPED ~~~~~~~~~~~~~')
        # print('box_loss:', box_loss)
        # print('angle_loss:', angle_loss)
        # print('object_loss:', object_loss)
        # print('no_object_loss:', no_object_loss)
        # print('class_loss:', class_loss)
        # print('reg_object_loss', reg_object_loss)

        # Total loss
        return (
            box_loss
            + angle_loss
            + object_loss
            + no_object_loss
            + class_loss
            # + reg_object_loss
        )
